# Star Jeans _ Product pricing project based on competition

## Business problem
- What is the best selling price for pants? 

## Questions to be answered:
- What is the best selling price for the pants?
- How many types of pants and their colors for the initial product?
- What are the raw materials needed to make the pants?

## Data source
- H&M: https://www2.hm.com/en_us/men/products/jeans.html
- Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

# Imports

In [88]:
# !pip install requests
# !pip install bs4

In [89]:
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np
import re


In [90]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
           
page = requests.get( url, headers=headers )

In [91]:
soup = BeautifulSoup( page.text, 'html.parser' )

In [92]:
products = soup.find( 'ul', class_='products-listing small' )

In [93]:
product_list = products.find_all( 'article', class_='hm-product-item')

# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]

In [94]:
data = pd.DataFrame( [product_id, product_category, product_name,product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name','product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

In [95]:
total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get('data-total')
total_item

'88'

In [96]:
page_number = np.ceil( int( total_item ) / 36 )
page_number

3.0

In [97]:
url02 = url + '?page-size=' + str( int( page_number*36 ) )
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

## One Product

In [98]:
# # API Requests
# url = 'https://www2.hm.com/en_us/productpage.0636207010.html'
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# page = requests.get( url, headers=headers )

# # Beautiful Soup object
# soup = BeautifulSoup( page.text, 'html.parser' )



In [99]:
# # ==================== color name =================================
# product_list = soup.find_all( 'a', class_='filter-option miniature' )
# color_name = [p.get( 'data-color' ) for p in product_list]

# # product id
# product_id = [p.get( 'data-articlecode' ) for p in product_list]
# df_color = pd.DataFrame( [product_id, color_name] ).T
# df_color.columns = ['product_id', 'color_name']

# # # generate style id + color id
# df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
# df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

# # ==================== composition =================================
# product_composition_list = soup.find_all( 'div',class_='pdp-description-list-item' )
# product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

# # rename dataframe
# df_composition = pd.DataFrame( product_composition ).T
# df_composition.columns = df_composition.iloc[0]

# # delete first row
# df_composition = df_composition.iloc[1:].fillna( method='ffill' )

# # generate style id + color id
# df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3])
# df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:])

# # merge data color + decomposition
# data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit','Composition']], how='left', on='style_id' )

## Multiple Products

In [100]:

# empty dataframe
df_details = pd.DataFrame()

# unique columns for all products
aux = []
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )

for i in range( len( data ) ):
    
# API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
        page = requests.get( url, headers=headers )

        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )

        # ==================== color name =================================
        product_list = soup.find_all( 'a', class_='filter-option miniature' )
        color_name = [p.get( 'data-color' ) for p in product_list]

        # product id
        product_id = [p.get( 'data-articlecode' ) for p in product_list]
        df_color = pd.DataFrame( [product_id, color_name] ).T
        df_color.columns = ['product_id', 'color_name']

        # generate style id + color id
        df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
        df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

# ==================== composition =================================

        product_composition_list = soup.find_all( 'div',class_='pdp-description-list-item' )
        product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # reaname dataframe
        df_composition = pd.DataFrame( product_composition ).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        # generate style id + color id
        df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3] )
        df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:] )
        aux = aux + df_composition.columns.tolist()

        # merge data color + decomposition
        data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit','Composition', 'Size', 'Product safety']], how='left', on='style_id' )

        # all details products
        df_details = pd.concat( [df_details, data_sku], axis=0 )

                                                                                                       
# Join Showroom data + details
data['style_id'] = data['product_id'].apply( lambda x: x[:-3] )
data['color_id'] = data['product_id'].apply( lambda x: x[-3:] )
data_raw = pd.merge( data, df_details[['style_id', 'color_name', 'Fit','Composition', 'Size', 'Product safety']], how = 'left', on='style_id')

data_raw.to_csv('Datasets/data_raw_star_jeans.csv')

## Clear Data

In [359]:
data = pd.read_csv('Datasets/data_raw_star_jeans.csv')
data.head()

,Unnamed: 0,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,Composition,Size,Product safety
0,0,985197001,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-20 20:04:08,985197,1,Midnight blue,Slim fit,Pocket lining: Cotton 100%,"The model is 189cm/6'2"" and wears a size 32/32",NaN
1,1,985197001,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-20 20:04:08,985197,1,Midnight blue,Slim fit,"Shell: Cotton 98%, Elastane 2%","The model is 189cm/6'2"" and wears a size 32/32",NaN
2,2,985197001,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-20 20:04:08,985197,1,Denim blue,Slim fit,Pocket lining: Cotton 100%,"The model is 189cm/6'2"" and wears a size 32/32",NaN
3,3,985197001,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-20 20:04:08,985197,1,Denim blue,Slim fit,"Shell: Cotton 98%, Elastane 2%","The model is 189cm/6'2"" and wears a size 32/32",NaN
4,4,985197001,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-20 20:04:08,985197,1,Dark denim blue,Slim fit,Pocket lining: Cotton 100%,"The model is 189cm/6'2"" and wears a size 32/32",NaN


In [360]:


# product id
data = data.dropna( subset=['product_id'] )
data['product_id'] = data['product_id'].astype( int )

# product name
data['product_name'] = data['product_name'].apply( lambda x: x.replace( ' ','_' ).lower() )

# product price
data['product_price'] = data['product_price'].apply( lambda x: x.replace( '$ ','' ) ).astype( float )

# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime( data['scrapy_datetime'],format='%Y-%m-%d %H:%M:%S' )

# style id
data['style_id'] = data['style_id'].astype( int )

# color id
data['color_id'] = data['color_id'].astype( int )

# color name
data['color_name'] = data['color_name'].apply( lambda x: x.replace( ' ', '_' ).replace( '/', '_' ).lower() 
                                              if pd.notnull( x ) else x )

# fit
data['Fit'] = data['Fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )

# size number
data['size_number'] = data['Size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0)
                                         if pd.notnull( x ) else x )

data['size_number'] = data['Size'].apply( lambda x: re.search( '\d+', x ).group(0) if pd.notnull( x ) else x )

# size model
data['size_model'] = data['Size'].str.extract( '(\d+/\\d+)' )
data = data.drop( columns=['Size', 'Product safety'], axis=1 )

# composition
### drop Pocket lining,Lining, Shell

data=data[~data['Composition'].str.contains('Pocket lining:',na=False)]
data=data[~data['Composition'].str.contains('Lining:',na=False)]
data=data[~data['Composition'].str.contains('Shell:',na=False)]
data=data[~data['Composition'].str.contains('Pocket:',na=False)]

# break composition by comma
df1=data['Composition'].str.split(',', expand=True)


##############################Creating a collumn for each matterials#####################################

# cotton/polyester/Elastano/Elasterell

df_ref= pd.DataFrame(index=np.arange(len(data)),columns=['cotton','polyester','elastane','elasterell'])

# Coton
df_cotton=df1[0]
df_cotton.name = 'cotton'

df_ref= pd.concat([df_ref,df_cotton],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 
    
# Polyester

df_polyester= df1.loc[df1[1].str.contains('Polyester',na=True),1]
df_polyester.name = 'polyester'

df_ref= pd.concat([df_ref,df_polyester],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 

# # Elastano
df_elastane= df1.loc[df1[1].str.contains('Elastane',na=True),1]
df_elastane.name = 'elastane'

df_elastane1=df1[2]

df_elastane=df_elastane.combine_first(df_elastane1)

df_ref= pd.concat([df_ref,df_elastane],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 



# Elasterell
df_elasterell= df1.loc[df1[1].str.contains('Elasterell',na=True),1]
df_elasterell.name = 'elasterell'

df_ref= pd.concat([df_ref,df_elasterell],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 


#Final join
data=pd.concat([data, df_ref], axis=1)
data['cotton']=data['cotton'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['elastane']=data['elastane'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['polyester']=data['polyester'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['elasterell']=data['elasterell'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)

In [362]:
data.dtypes

Unnamed: 0                 float64
product_id                 float64
product_category            object
product_name                object
product_price              float64
scrapy_datetime     datetime64[ns]
style_id                   float64
color_id                   float64
color_name                  object
Fit                         object
Composition                 object
size_number                 object
size_model                  object
cotton                     float64
polyester                  float64
elastane                   float64
elasterell                 float64
dtype: object

In [252]:
df1[2].unique()

array([None, ' Elastane 2%', ' Elastane 1%', nan], dtype=object)